In [4]:
import pandas as pd
import numpy as np
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError

In [38]:
train = pd.read_csv('Data/sales_train_evaluation.csv')
cal = pd.read_csv('Data/calendar.csv')
prices = pd.read_csv('Data/sell_prices.csv')

In [15]:
# range starting from 6 is for the initial 6 non-demand columns
def add_naive_mse(df,headers):
    naive_mse = pd.DataFrame(data={'naive_mse':list(np.repeat(0,df.shape[0]))})
    for i in range(headers,len(df.columns)-1):
        new_naive_mse = (df[df.columns[i+1]] - df[df.columns[i]])**2
        naive_mse['naive_mse'] += new_naive_mse
    return naive_mse
    

In [89]:
def create_levels(df):
    #total
    full_wlevels = pd.DataFrame(data={'level':['total'],'id':['total'],'item_id':['total'],'dept_id':['total'],'cat_id':['total'],'store_id':['total'],'state_id':['total']})
    full_wlevels = pd.concat([full_wlevels,df.sum(numeric_only=True).to_frame().T],axis=1)
    #state
    new_level = df.groupby('state_id').sum().reset_index()
    new_level['level'] = 'state'
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #store
    new_level = df.groupby('store_id').sum().reset_index()
    new_level['level'] = 'store'
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #category
    new_level = df.groupby('cat_id').sum().reset_index()
    new_level['level'] = 'category'
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #department
    new_level = df.groupby('dept_id').sum().reset_index()
    new_level['level'] = 'department'
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #state_category
    new_level = df.groupby(['state_id','cat_id']).sum().reset_index()
    new_level['level'] = 'state_category'
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #state_department
    new_level = df.groupby(['state_id','dept_id']).sum().reset_index()
    new_level['level'] = 'state_department'
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #store_category
    new_level = df.groupby(['store_id','cat_id']).sum().reset_index()
    new_level['level'] = 'store_category'
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #store_department
    new_level = df.groupby(['store_id','dept_id']).sum().reset_index()
    new_level['level'] = 'store_department'
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #item
    new_level = df.groupby('item_id').sum().reset_index()
    new_level['level'] = 'item'
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #item_state
    new_level = df.groupby(['item_id','state_id']).sum().reset_index()
    new_level['level'] = 'item_state'
    full_wlevels = pd.concat([full_wlevels,new_level],axis=0)
    #item_store
    df['level'] = 'item_store'
    full_wlevels = pd.concat([full_wlevels,df],axis=0)
    #return final df
    return full_wlevels

In [8]:
full_wlevels = create_levels(train).reset_index(drop=True)

In [9]:
full_wlevels

,level,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,total,total,total,total,total,total,total,32631,31749,23783,...,44073,54308,59921,42362,38777,37096,36963,42552,51518,54338
1,state,NaN,NaN,NaN,NaN,NaN,CA,14195,13805,10108,...,18471,23037,24704,17721,16150,15678,16297,17430,23103,24644
2,state,NaN,NaN,NaN,NaN,NaN,TX,9438,9630,6778,...,12320,12366,16249,12228,11370,10375,9162,12303,13681,14815
3,state,NaN,NaN,NaN,NaN,NaN,WI,8998,8314,6897,...,13282,18905,18968,12413,11257,11043,11504,12819,14734,14879
4,store,NaN,NaN,NaN,NaN,CA_1,NaN,4337,4155,2816,...,4951,6245,6707,4568,3949,3995,4136,4433,5764,6289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42835,item_store,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,...,1,0,3,0,1,1,0,0,1,1
42836,item_store,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,...,0,0,0,0,0,0,1,0,1,0
42837,item_store,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,...,0,0,1,2,0,1,0,1,0,2
42838,item_store,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [20]:
trainf = full_wlevels.iloc[:,:1920]
testf = full_wlevels.iloc[:,np.r_[0:7,1920:1948]]

In [23]:
trainf['naive_mse'] = add_naive_mse(trainf,7)

In [78]:
test_weights = train.iloc[:,np.r_[0:6,1919:1947]]
test_weights = pd.melt(test_weights, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
cal_M = cal[['wm_yr_wk','d']]
test_weights = pd.merge(test_weights, cal_M, how = 'left', left_on = ['day'], right_on = ['d']).drop('d', axis=1)
test_weights = test_weights.merge(prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
test_weights['sales'] = test_weights['demand']*test_weights['sell_price']
test_weights = test_weights.drop(['demand','wm_yr_wk','sell_price'],axis=1)
test_weights = test_weights.set_index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','day'])['sales'].unstack().reset_index()

In [90]:
test_sales_wlevels = create_levels(test_weights)

In [103]:
total_sales = float(test_sales_wlevels[test_sales_wlevels['level']=='total'].sum(numeric_only=True,axis=1))

In [116]:
test_sales_wlevels['weight'] = test_sales_wlevels.sum(numeric_only=True,axis=1)/total_sales/10

In [117]:
test_sales_wlevels

,level,id,item_id,dept_id,cat_id,store_id,state_id,d_1914,d_1915,d_1916,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,weight
0,total,total,total,total,total,total,total,123361.97,112650.08,107784.43,...,171719.78,191941.73,134870.66,122419.42,116333.11,117456.32,134000.67,162395.65,175297.42,1.000000e-01
0,state,NaN,NaN,NaN,NaN,NaN,CA,57761.95,50120.99,48522.10,...,76920.38,82372.48,59146.01,52456.78,51584.11,53695.76,56724.33,76344.32,82268.63,4.413755e-02
1,state,NaN,NaN,NaN,NaN,NaN,TX,34902.54,32188.39,30763.33,...,40080.81,54031.71,40042.38,36611.47,33051.07,30074.18,39795.89,44046.80,49441.48,2.759605e-02
2,state,NaN,NaN,NaN,NaN,NaN,WI,30697.48,30340.70,28499.00,...,54718.59,55537.54,35682.27,33351.17,31697.93,33686.38,37480.45,42004.53,43587.31,2.826640e-02
0,store,NaN,NaN,NaN,NaN,CA_1,NaN,13887.08,11996.21,12037.84,...,20314.35,21730.28,15236.62,12447.15,12920.62,13259.90,13999.65,18637.70,21110.44,1.112623e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,item_store,HOUSEHOLD_2_516_TX_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,TX_2,TX,0.00,0.00,0.00,...,0.00,0.00,5.94,0.00,0.00,0.00,0.00,0.00,5.94,6.090784e-07
30486,item_store,HOUSEHOLD_2_516_TX_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,TX_3,TX,0.00,0.00,0.00,...,5.94,0.00,0.00,0.00,0.00,0.00,5.94,5.94,11.88,1.243535e-06
30487,item_store,HOUSEHOLD_2_516_WI_1_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_1,WI,5.94,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,5.94,0.00,0.00,7.613479e-07
30488,item_store,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,0.00,0.00,0.00,...,0.00,5.94,5.94,0.00,0.00,0.00,0.00,0.00,0.00,3.045392e-07
